In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def simulate_nonlinear_system(x, u, disturbance):
    """Simulate a nonlinear system with disturbance.
    Args:
        x: Current state of the system.
        u: Control input.
        disturbance: Random disturbance added to the system.
    Returns:
        ndarray: Derivative of the system state.
    """
    dx1 = x[1]
    dx2 = -x[0] + u + disturbance  # Add disturbance to introduce randomness
    return np.array([dx1, dx2])

In [ ]:
def luenberger_observer(x_hat, u, y, A, C, L):
    """Simulate a Luenberger observer for state estimation.
    Args:
        x_hat: Estimated state.
        u: Control input.
        y: Measurement output.
        A: System matrix.
        C: Output matrix.
        L: Observer gain matrix.
    Returns:
        ndarray: Derivative of the estimated state.
    """
    x_hat_dot = A.dot(x_hat) + L.dot(y - C.dot(x_hat)).T.ravel() + B * u
    return x_hat_dot

In [ ]:
# Simulation parameters
t_max = 50.0
dt = 0.01
u = 1  # Control input
time = np.arange(0, t_max, dt)
x_true = np.zeros((len(time), 2))
x_hat = np.zeros((len(time), 2))

# Initial states
x_true[0] = np.array([1.0, 0.0])
x_hat[0] = np.array([0.9, 0.0])

# Luenberger observer parameters
A = np.array([[0, 1], [-1, 0]])
B = np.array([0, 1])
C = np.array([1, 0])
L = np.array([[0.5], [0.5]])  # Gain matrix for the observer

# Run simulation
for i in range(1, len(time)):
    disturbance = np.random.normal(0, 0.01)
    x_true[i] = x_true[i-1] + simulate_nonlinear_system(x_true[i-1], u, disturbance) * dt

    # State estimation using Luenberger observer
    y = C.dot(x_true[i])  # Measurement
    x_hat_dot = luenberger_observer(x_hat[i-1], u, y, A, C, L)
    x_hat[i] = x_hat[i-1] + x_hat_dot * dt

# Plotting
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(time, x_true[:, 0], label='True State (x1)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, x_true[:, 0], label='True State (x1)')
plt.plot(time, x_hat[:, 0], label='Estimated State (x1)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.savefig('x1.pdf', format='pdf')
plt.close()

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(time, x_true[:, 1], label='True State (x2)')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, x_true[:, 1], label='True State (x2)')
plt.plot(time, x_hat[:, 1], label='Estimated State (x2)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.show()
plt.close()